In [2]:
! pip install tqdm
! pip install regex

In [68]:
import json

import bz2
import regex
from tqdm import tqdm
from scipy import sparse

In [69]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


C:\Users\Python\Anaconda3\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['text']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [7]:
responses = []
with bz2.BZ2File('../data/banki_responses.json.bz2', 'r') as thefile:
    for row in tqdm(thefile):
        resp = json.loads(row)
        if not resp['rating_not_checked'] and (len(resp['text'].split()) > 0):
            responses.append(resp)

FileNotFoundError: ignored

In [8]:
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# choose a local (colab) directory to store the data.
local_download_path = os.path.expanduser('~/data')
try:
  os.makedirs(local_download_path)
except: pass

# 2. Auto-iterate using the query syntax
#    https://developers.google.com/drive/v2/web/search-parameters
file_list = drive.ListFile(
    {'q': "'1UdMM6GRG706jvagUmAzwMIahcUKwj9Fr' in parents and trashed=false"}).GetList()

for f in file_list:
  # 3. Create & download by id.
  print('title: %s, id: %s' % (f['title'], f['id']))
  fname = os.path.join(local_download_path, f['title'])
  print('downloading to {}'.format(fname))
  f_ = drive.CreateFile({'id': f['id']})
  f_.GetContentFile(fname)


responses = []
with bz2.BZ2File(fname, 'r') as thefile:
    for row in tqdm(thefile):
        resp = json.loads(row)
        if not resp['rating_not_checked'] and (len(resp['text'].split()) > 0):
            responses.append(resp)
        

title: HW2_banki_sent-vocab.ipynb, id: 1fSDUbtpAqvTQm95dR9jxjbf2sOpj31lx
downloading to /content/data/HW2_banki_sent-vocab.ipynb
title: banki_responses.json.bz2, id: 1Pmg5qoggsx3xt5VUyqpGnIJUF-gR61qZ
downloading to /content/data/banki_responses.json.bz2


201030it [02:08, 1567.74it/s]


#  Домашнее задание по NLP # 2 [100 баллов] 
## Составление словарей для классификации по тональности
При классификации текстов или предложений по тональности необходимо использовать оценочные словари для предметной области, то есть, такие словари, в которых содержатся отрицательные и позитивные слова для какой-то предметной области. Идея подобных словарей основана на следующих наблюдениях: во-первых, для разных товаров используются разные оценочные слова (например бывает “захватывающая книга”, но не бывает “захватывающих лыж”), во-вторых, в контексте разных товаров одни и те же слова могут иметь разную окраску (слово “тормоз” в отзыве на велосипед имеет нейтральную окраску, в отзыве на компьютер – резко негативную, “пыль” в контексте пылесосов – нейтральную, в контексте кофемолок – положительную (“мелкий помол в пыль”)). Еще один пример: "теплое пиво" – это плохо, а "теплый свитер" – это хорошо.  

Составление таких словарей в ручную – трудоемкий процесс, но, к счастью, его не сложно автоматизировать, если собрать достаточно большие корпуса отзывов. В этом домашнем задании вам предстоит попробовать реализовать один их подходов к составлению оценочных словарей, основанный на статье Inducing Domain-Specific Sentiment Lexicons from Unlabeled Corpora (https://nlp.stanford.edu/pubs/hamilton2016inducing.pdf).


Данные для задания – уже знакомые вам отзывы на банки, собранные с нескольких сайтов Рунета. Отзывы могут быть как положительными (оценка 5), так и отрицательными (оценка 1).

In [9]:
responses[99]['text']

'Здравствуйте! Хотела написать, что мне месяц не выдают карту ко вкладу, ссылаясь на "нам же их из Самары везут" (на секундочку 5 часов езды от нашего города). Но! Прочитала, что людям 3,5 месяцев не выдают карту, и поняла, что у меня все хорошо, пока что. И подарок мне дали, и кулер в отделении есть. Так что я, конечно, готова ждать. Правда хотелось бы не очень долго.'

## Часть 1. Обучение модели word2vec [50 баллов]

1. Разбейте всю коллекцию отзывов на предложения. Лемматизируйте все слова. 
2. Обучите по коллекции предложений word2vec
3. Приведите несколько удачных и неудачных примеров решения стандартных текстов для word2vec:
    * тест на определение ближайших слов
    * тест на аналогии (мужчина – король : женщина – королева)
    * тест на определение лишнего слова.
    
4. Постройте несколько визуализаций:
    * TSNE для топ-100 (или топ-500) слов и найдите осмысленные кластеры слов
    * задайте координаты для нового пространства следующим образом: одна  ось описывает отношение "плохо – хорошо", вторая – "медленно – быстро" и найдите координаты названий банков в этих координатах.  Более формально:
    берем вектор слова "хорошо", вычитаем из него вектор слова "плохо", получаем новый вектор, который описывает разницу между хорошими и плохими словами. Берем вектор слова "сбербанк" и умножаем его на этот новый вектор – получаем координату по первой оси. Аналогично – для второй оси. Две координаты уже можно нарисовать на плоскости.  
 


Ссылка на примеры визуализаций: https://towardsdatascience.com/game-of-thrones-word-embeddings-does-r-l-j-part-2-30290b1c0b4b

In [15]:
responses[0]

{'author': 'uhnov1',
 'bank_license': 'лицензия № 2562',
 'bank_name': 'Бинбанк',
 'city': 'г. Москва',
 'datetime': '2015-06-08 12:50:54',
 'num_comments': 0,
 'rating_grade': None,
 'rating_not_checked': False,
 'text': 'Добрый день! Я не являюсь клиентом банка и поручителем по кредитному договору, а также не являюсь каким-либо другим лицом, письменно  оформившим отношения с банком по поводу урегулирования чьей-либо  задолженности.  Начиная с марта 2015 года начали приходить бесконечные письма из ООО "Примо коллект"на мой адрес: город Москва, Уваровский переулок, дом 10, квартира 111, с угрозами о возбуждении уголовного дела в отношении гражданина Филиппова Эдуарда Владимировича, который уклоняется от уплаты взятых им кредитов: договор № 81014 от 20.10.2013 года и договор № 2464946 от 09.10.2014 года. Со всей ответственностью\xa0 хочу Вас заверить, что вышеуказанный гражданин, которого Вы разыскиваете, мне не знаком и никогда в моем адресе не был зарегистрирован. Каким образом Вы не 

In [11]:
texts = []

for i in range(len(responses)):
    texts.append(responses[i]['text'])
        
  

In [41]:
texts[0]

'Добрый день! Я не являюсь клиентом банка и поручителем по кредитному договору, а также не являюсь каким-либо другим лицом, письменно  оформившим отношения с банком по поводу урегулирования чьей-либо  задолженности.  Начиная с марта 2015 года начали приходить бесконечные письма из ООО "Примо коллект"на мой адрес: город Москва, Уваровский переулок, дом 10, квартира 111, с угрозами о возбуждении уголовного дела в отношении гражданина Филиппова Эдуарда Владимировича, который уклоняется от уплаты взятых им кредитов: договор № 81014 от 20.10.2013 года и договор № 2464946 от 09.10.2014 года. Со всей ответственностью\xa0 хочу Вас заверить, что вышеуказанный гражданин, которого Вы разыскиваете, мне не знаком и никогда в моем адресе не был зарегистрирован. Каким образом Вы не удостоверившись в подлинности его документов оформили на его имя кредитный договор, мне по меньшей мере не понятно,\xa0 и почему по Вашей милости я должна переживать и бояться за себе и свое имущество. Письма доставляют мн

In [12]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

text_sents= []

for text in texts:
    
    for sent in sent_tokenize(text):
      text_sents.append(sent)
      
print(len(text_sents))


[nltk_data] Downloading package punkt to /content/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2664456


In [13]:
text_sents[0]

'Добрый день!'

In [14]:
! pip install pymystem3

In [76]:

from pymystem3 import Mystem
import re
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

m = Mystem()


regex = re.compile("А-Яа-я:=!\)\()A-z\_\%/|+")

def words_only(text, regex=regex):
    try:
        return " ".join(regex.findall(text))
    except:
        return ""



def lemmatize(text, mystem=m):
    try:
        return "".join(m.lemmatize(text)).strip()  
    except:
        return " "


error: unbalanced parenthesis at position 13

In [16]:
sents_words_only = []
for sent in text_sents:
    
    sents_words_only.append(words_only(sent))


In [42]:
from tqdm import tqdm
sents_lemm = []
i=0
for text in tqdm(sents_words_only):
    sents_lemm.append(lemmatize(text,m).split())
    

100%|██████████| 2664456/2664456 [34:33<00:00, 1285.18it/s]


In [19]:
m.lemmatize(sents_words_only[0])

['добрый', ' ', 'день', '!', '\n']

In [43]:
sents_lemm[1]

['я',
 'не',
 'являться',
 'клиент',
 'банк',
 'и',
 'поручитель',
 'по',
 'кредитный',
 'договор',
 'а',
 'также',
 'не',
 'являться',
 'какой',
 'либо',
 'другой',
 'лицо',
 'письменно',
 'оформлять',
 'отношение',
 'с',
 'банк',
 'по',
 'повод',
 'урегулирование',
 'чей',
 'либо',
 'задолженность']

In [35]:
sents_lemm[8]

'кроме это быть подавать заявление в полиция и заявление в суд возмещение причинять я моральный вред'

In [56]:
import codecs
with codecs.open('lemm.txt', "r",encoding='utf-8', errors='ignore') as fdata:
for item in fdata:
    thefile.write("%s\n" % item)


IndentationError: expected an indented block (<ipython-input-56-00fc828f000e>, line 3)

In [71]:


sents_lemm = []

# open file and read the content in a list
import codecs
with codecs.open('lemm.txt', "r",encoding='utf-8', errors='ignore') as filehandle:
    for line in filehandle:
        # remove linebreak which is the last character of the string
        currentPlace = line[:-1]

        # add item to the list
        sents_lemm.append(currentPlace)

In [81]:
sents_lemm2 = []
for sent in sents_lemm:
    
    sents_lemm2.append(words_only(sent))


In [83]:
sents_lemm3 = [i.split()  for i in sents_lemm2]

In [84]:
sents_lemm3[1]

['[',
 'я',
 'не',
 'являться',
 'клиент',
 'банк',
 'и',
 'поручитель',
 'по',
 'кредитный',
 'договор',
 'а',
 'также',
 'не',
 'являться',
 'какой',
 'либо',
 'другой',
 'лицо',
 'письменно',
 'оформлять',
 'отношение',
 'с',
 'банк',
 'по',
 'повод',
 'урегулирование',
 'чей',
 'либо',
 'задолженность',
 ']']

In [0]:
thefile.close()

In [22]:
! pip install gensim

    100% |████████████████████████████████| 22.6MB 1.5MB/s 
    100% |████████████████████████████████| 1.4MB 7.5MB/s 
    100% |████████████████████████████████| 133kB 18.3MB/s 
    100% |████████████████████████████████| 61kB 18.8MB/s 
    100% |████████████████████████████████| 4.4MB 7.0MB/s 
    100% |████████████████████████████████| 552kB 20.1MB/s 
  Running setup.py bdist_wheel for smart-open ... - \ done
  Stored in directory: /content/.cache/pip/wheels/73/f1/9b/ccf93d4ba073b6f79b1ed9df68ab5ce048d8136d0efcf90b30
  Running setup.py bdist_wheel for bz2file ... - done
  Stored in directory: /content/.cache/pip/wheels/81/75/d6/e1317bf09bf1af5a30befc2a007869fa6e1f516b8f7c591cb9
Successfully built smart-open bz2file


In [3]:
  %%time
from gensim.models import Word2Vec
model = Word2Vec(sents_lemm, size=100, window=3, min_count=5, workers=32)
model.save("sent_w2v.model")

C:\Users\Python\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


NameError: name 'sents_lemm' is not defined

In [4]:
model = Word2Vec.load("sent_w2v.model") 

In [0]:
from google.colab import files
files.download('sent_w2v.model')

In [6]:
vocab_size = len(model.wv.vocab)
print(model.wv.index2word[vocab_size - 1])

(надувательство


In [8]:
model.most_similar("банк")

C:\Users\Python\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('банка', 0.8276962041854858),
 ('банк)', 0.8032518625259399),
 ('банк!', 0.7613455057144165),
 ('сбербанк', 0.6706583499908447),
 ('ситибанк', 0.6400394439697266),
 ('банка)', 0.6363973617553711),
 ('росбанк', 0.602057695388794),
 ('сбера', 0.5882582664489746),
 ('БРС', 0.5836309194564819),
 ('сбер', 0.5561813712120056)]

In [13]:
model.most_similar(positive=["сбербанк","очередь"], negative=["плохо"])

C:\Users\Python\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('мкб', 0.5362464189529419),
 ('очередь)', 0.5252681970596313),
 ('СБ', 0.46801820397377014),
 ('ситибанк', 0.465644508600235),
 ('очередь!', 0.44585591554641724),
 ('сбера', 0.43524935841560364),
 ('очередь:', 0.39914894104003906),
 ('ПСБ', 0.3979168236255646),
 ('окно', 0.3935149013996124),
 ('бинбанк', 0.38939368724823)]

In [10]:
model.doesnt_match("борщ сметана макароны пирожок консомэ кошка".split())

C:\Users\Python\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.


'пирожок'

In [99]:
model.doesnt_match("банк деньги клиент государство сбербанк".split())

C:\Users\Python\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.


'деньги'

In [89]:
top_words = []
from nltk import FreqDist
fd = FreqDist()
for text in sents_lemm3:
    fd.update(text)
for i in fd.most_common(1000):
    top_words.append(i[0])
print(top_words)

[']', '[', 'в', 'я', 'и', 'не', 'на', 'что', 'банк', 'с', 'быть', 'по', 'карта', 'то', 'мой', 'это', 'а', 'как', 'но', 'этот', 'за', 'к', 'у', 'деньги', 'о', 'день', 'который', 'сотрудник', 'кредит', 'так', 'вы', 'клиент', 'весь', 'от', 'отделение', 'они', 'свой', 'счет', 'мочь', 'сумма', 'для', 'же', 'она', 'из', 'год', 'вклад', 'до', 'сказать', 'такой', 'уже', 'получать', 'он', 'заявление', 'при', 'все', 'данный', 'через', 'договор', 'ваш', 'т', 'время', 'мы', 'номер', 'какой', 'вопрос', 'раз', 'только', 'кредитный', 'нет', 'платеж', 'если', 'телефон', 'приходить', 'другой', 'средство', 'тот', 'г', 'после', 'еще', 'обращаться', 'один', 'ответ', 'звонок', 'офис', 'звонить', ')', 'сбербанк', 'когда', 'бы', 'рубль', 'со', 'ни', 'или', 'написать', 'проблема', 'чтобы', 'решать', 'месяц', 'очень', 'позвонить', 'ситуация', 'без', 'человек', 'работа', 'информация', 'смс', 'отвечать', 'операция', 'работать', 'почему', 'документ', 'просить', 'услуга', 'хотеть', 'давать', 'сообщать', 'должный',

In [96]:
top_words_vec = model[top_words]

C:\Users\Python\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [97]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
top_words_tsne = tsne.fit_transform(top_words_vec)

In [98]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=top_words_tsne[:,0],
                                    x2=top_words_tsne[:,1],
                                    names=top_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

Loading BokehJS ...

In [107]:
from sklearn import datasets
from sklearn.semi_supervised import label_propagation

from sklearn.metrics import confusion_matrix, classification_report

digits = datasets.load_digits()
rng = np.random.RandomState(0)
indices = np.arange(len(digits.data))
rng.shuffle(indices)

X = digits.data[indices[:330]]
y = digits.target[indices[:330]]

In [118]:
y=numpy.array([numpy.array(xi) for xi in x])

numpy.ndarray

## Часть 2. Распространение метки [50 баллов]

Определите 5-8 позитивных слов (например, “быстрый”, “удобный”) и 5-8  негативных слов (например,“очередь”, “медленно”). Эти слова будут основной будущего оценочного словаря. Пусть позитивному классу соответствует метка 1, негативному – -1. Пометьте выбранные слова в лексическом графе соответствующими метками. Запустите любой известный вам метод распространения метки (Label Propogation) в лексическом графе. На выходе метода распространения ошибки должны быть новые слова, помеченные метками 1 и -1 – это и есть искомые оценочные слова.

Алгоритмы распространения метки устроены примерно так: пусть мы находимся в выршине, помеченном +1. С какой-то вероятностью мы переносим эту метку на соседние узлы. С меньшей вероятностью переносим ее на вершины на расстоянии два. В конце распространения метки, часть вершин оказывается помечена меткой +1, часть – -1, большая часть остается без метки.

Рекомендуемые алгоритмы распространения метки:
1. ```graphlab.label_propagation``` (```graphlab``` доступен бесплатно по образовательной лицензии)
2. ```sklearn.semi_supervised.LabelPropagation``` 
3. ```sklearn.semi_supervised.LabelSpreading```

In [100]:
from sklearn.semi_supervised import label_propagation
from sklearn.metrics import confusion_matrix, classification_report

In [101]:
X_train = ['положительный','благодарность','одобрять','можно','вклад','заблокировать','угрожать','блокировка','нарушение','неприятный']
y_train = [1,1,1,1,1,-1,-1,-1,-1,-1]

In [126]:
x = model[X_train]
y = np.array (y_train)

C:\Users\Python\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [131]:
model[model.wv.vocab]

C:\Users\Python\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([[ 1.78555882e+00,  7.93943644e-01,  1.50274467e+00, ...,
         3.05874228e+00,  3.50992769e-01, -2.41165924e+00],
       [ 1.29047501e+00, -3.00577307e+00, -7.08908617e-01, ...,
        -1.28014648e+00,  7.82221079e-01,  3.81452531e-01],
       [ 2.19577670e-01,  3.53595495e-01, -2.92775226e+00, ...,
        -4.06138450e-01, -3.13589001e+00, -1.60593182e-01],
       ...,
       [ 6.38936600e-03, -3.63482162e-02,  1.14630364e-01, ...,
         2.54743360e-02, -2.60709729e-02, -1.01988092e-01],
       [ 1.25766676e-02, -3.64708006e-02,  1.44111037e-01, ...,
         5.78806698e-02, -1.68523062e-02, -9.47405770e-02],
       [ 2.52768155e-02, -3.03440746e-02,  8.76581669e-02, ...,
        -3.56098812e-04, -3.07984669e-02, -1.80557191e-01]], dtype=float32)

In [ ]:
word_box = []
for sent in sents_lemm3:
    for word in sent:
        word_box.append(word)
        

In [102]:
lp_model = label_propagation.LabelSpreading(gamma=0.25, max_iter=5)

In [127]:
lp_model.fit(x, y)

C:\Users\Python\Anaconda3\lib\site-packages\sklearn\semi_supervised\label_propagation.py:288: ConvergenceWarning: max_iter=5 was reached without convergence.
  category=ConvergenceWarning


LabelSpreading(alpha=0.2, gamma=0.25, kernel='rbf', max_iter=5, n_jobs=1,
        n_neighbors=7, tol=0.001)

In [132]:
predicted_labels = lp_model.predict(model[model.wv.vocab])

C:\Users\Python\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.
C:\Users\Python\Anaconda3\lib\site-packages\sklearn\semi_supervised\label_propagation.py:201: RuntimeWarning: invalid value encountered in true_divide
  probabilities /= normalizer


In [133]:
predicted_labels

array([1, 1, 1, ..., 1, 1, 1])